In [22]:
import pandas  as pd
import seaborn as sea
import matplotlib.pyplot as plt
import glob
paths = sorted(glob.glob(f'datasets/Cryptocurrency Historical Prices/*.csv'))

In [3]:
dicionario = {'Csv':[],'Instancias':[]}
for i in range(len(paths)):
    dicionario['Csv'].append(paths[i].split('/')[2])
    dicionario['Instancias'].append(pd.read_csv(paths[i]).shape[0])

In [4]:
grupos = pd.DataFrame(dicionario)
grupos

,Csv,Instancias
0,coin_Aave.csv,275
1,coin_BinanceCoin.csv,1442
2,coin_Bitcoin.csv,2991
3,coin_Cardano.csv,1374
4,coin_ChainLink.csv,1385
5,coin_Cosmos.csv,845
6,coin_CryptocomCoin.csv,935
7,coin_Dogecoin.csv,2760
8,coin_EOS.csv,1466
9,coin_Ethereum.csv,2160


In [8]:
g1 = grupos.loc[grupos['Instancias']>2000]
g1

,Csv,Instancias
2,coin_Bitcoin.csv,2991
7,coin_Dogecoin.csv,2760
9,coin_Ethereum.csv,2160
11,coin_Litecoin.csv,2991
12,coin_Monero.csv,2602
13,coin_NEM.csv,2288
16,coin_Stellar.csv,2527
17,coin_Tether.csv,2318
22,coin_XRP.csv,2893


In [9]:
g2 = grupos.loc[(grupos['Instancias']>800) & (grupos['Instancias']<1000)] 
g2

,Csv,Instancias
5,coin_Cosmos.csv,845
6,coin_CryptocomCoin.csv,935
21,coin_WrappedBitcoin.csv,888


In [7]:
g3 = grupos.loc[(grupos['Instancias']<800)] 
g3

,Csv,Instancias
0,coin_Aave.csv,275
14,coin_Polkadot.csv,320
15,coin_Solana.csv,452
20,coin_Uniswap.csv,292


Farei a divisão de 3 grupos 
1. $>$ 2000 instancias
2. $>$ 800 & $<$ 1000 instancias
3. $<$ 800
> Para eviter viés

In [19]:
def Concats( paths , gn ):
    data = pd.DataFrame()
    for path in paths:
        for nomes in gn:
            if nomes in path:
                data =pd.concat([data, pd.read_csv(path)], ignore_index=True)
    return data
        

In [35]:
dataset = Concats(paths,g1['Csv'].values)
dataset.head(3)

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Bitcoin,BTC,2013-04-29 23:59:59,147.488007,134.000000,134.444,144.539993,0.0,1.603769e+09
1,2,Bitcoin,BTC,2013-04-30 23:59:59,146.929993,134.050003,144.000,139.000000,0.0,1.542813e+09
2,3,Bitcoin,BTC,2013-05-01 23:59:59,139.889999,107.720001,139.000,116.989998,0.0,1.298955e+09


&nbsp;

In [51]:
from sklearn.preprocessing   import LabelEncoder,StandardScaler
from sklearn.base            import BaseEstimator,TransformerMixin
from sklearn.pipeline        import Pipeline
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    cross_val_score,
    KFold
    )

## Pré-processamento

In [38]:
class Padronizacao(BaseEstimator,TransformerMixin):
    def __init__(self,columns:list | str) -> None:
        self.columns = columns 

    def fit(self,X,y=None):
        return self
    
    def transform(self, X:pd.DataFrame):  
        if ( type(self.columns) == list ):
            X[self.columns]   = StandardScaler().fit_transform(X[self.columns])
        else:
            X[[self.columns]] = StandardScaler().fit_transform(X[[self.columns]])
        return X
    
class Dummies(BaseEstimator,TransformerMixin):
    def __init__(self,columns:list | str) -> None:
        self.columns = columns 

    def fit(self,X,y=None):
        return self
    
    def transform(self, X:pd.DataFrame):
        return pd.get_dummies(X,columns=self.columns,dtype=int)

In [39]:
class RemovendoAtributos(BaseEstimator,TransformerMixin):
    def __init__(self,columns:list) -> None:
        self.columns = columns

    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        return X.drop(columns=self.columns,axis=0)

In [55]:
pipeline = Pipeline(
    steps=[
        ('Removing',RemovendoAtributos(['SNo','Name'])),
        ('Dummies ',Dummies(['Symbol']))
        ]
)
data = pipeline.fit_transform(dataset)

In [56]:
data

,Date,High,Low,Open,Close,Volume,Marketcap,Symbol_BTC,Symbol_DOGE,Symbol_ETH,Symbol_LTC,Symbol_USDT,Symbol_XEM,Symbol_XLM,Symbol_XMR,Symbol_XRP
0,2013-04-29 23:59:59,147.488007,134.000000,134.444000,144.539993,0.000000e+00,1.603769e+09,1,0,0,0,0,0,0,0,0
1,2013-04-30 23:59:59,146.929993,134.050003,144.000000,139.000000,0.000000e+00,1.542813e+09,1,0,0,0,0,0,0,0,0
2,2013-05-01 23:59:59,139.889999,107.720001,139.000000,116.989998,0.000000e+00,1.298955e+09,1,0,0,0,0,0,0,0,0
3,2013-05-02 23:59:59,125.599998,92.281898,116.379997,105.209999,0.000000e+00,1.168517e+09,1,0,0,0,0,0,0,0,0
4,2013-05-03 23:59:59,108.127998,79.099998,106.250000,97.750000,0.000000e+00,1.085995e+09,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23525,2021-07-02 23:59:59,0.667287,0.634726,0.659890,0.656763,2.061607e+09,3.030759e+10,0,0,0,0,0,0,0,0,1
23526,2021-07-03 23:59:59,0.683677,0.644653,0.655639,0.672888,1.872820e+09,3.105172e+10,0,0,0,0,0,0,0,0,1
23527,2021-07-04 23:59:59,0.707783,0.665802,0.673218,0.694945,1.885242e+09,3.206960e+10,0,0,0,0,0,0,0,0,1
23528,2021-07-05 23:59:59,0.695653,0.648492,0.695653,0.654300,2.076373e+09,3.019395e+10,0,0,0,0,0,0,0,0,1
